In [12]:
# --- Imports ---
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import json

# --- 1. Load saved model and scaler ---
model = load_model("success_predictor_model_optimized.keras", compile=False)

scaler = joblib.load("scaler.pkl")

# Load y_min and y_max for denormalizing the prediction
with open("y_norm_range.json") as f:
    y_min, y_max = json.load(f)

# --- 2. Define new event inputs ---


new_event_numeric = pd.DataFrame([
    # 🌟 GREAT — prime venue, top artist, high engagement
    {
        "rating": 4.9,
        "total_events": 60,
        "avg_checkins": 30.0,
        "avg_likes": 58.0,
        "avg_event_rating": 4.7,
        "median_event_rating": 4.8,
        "venue_popularity_tier": 5,
        "checkin_count": 58,
        "like_count": 96,
        "rating_avg": 4.9,
        "rating_count": 45,
        "engagement": 225.0,
        "event_weekday": 6,      # Saturday
        "event_month": 12,       # December (holiday season)
        "start_hour": 20,
        "price_rating_encoded": 2,
    },

    # 🌟 GREAT — strong local fan base + consistent quality
    {
        "rating": 4.6,
        "total_events": 50,
        "avg_checkins": 26.5,
        "avg_likes": 52.0,
        "avg_event_rating": 4.5,
        "median_event_rating": 4.5,
        "venue_popularity_tier": 4,
        "checkin_count": 50,
        "like_count": 90,
        "rating_avg": 4.7,
        "rating_count": 38,
        "engagement": 200.0,
        "event_weekday": 5,      # Friday
        "event_month": 6,
        "start_hour": 21,
        "price_rating_encoded": 2,
    },

    # 🌟 GREAT — popular theme, artist with viral past, great timing
    {
        "rating": 4.8,
        "total_events": 48,
        "avg_checkins": 27.0,
        "avg_likes": 50.0,
        "avg_event_rating": 4.4,
        "median_event_rating": 4.5,
        "venue_popularity_tier": 5,
        "checkin_count": 52,
        "like_count": 88,
        "rating_avg": 4.8,
        "rating_count": 40,
        "engagement": 215.0,
        "event_weekday": 6,
        "event_month": 9,
        "start_hour": 22,
        "price_rating_encoded": 2,
    },
])[[
    'rating', 'total_events', 'avg_checkins', 'avg_likes', 'avg_event_rating',
    'median_event_rating', 'venue_popularity_tier', 'checkin_count', 'like_count',
    'rating_avg', 'rating_count','engagement', 'event_weekday', 'event_month', 'start_hour',
    'price_rating_encoded'
]]






# 🏷️ Tag IDs (must be padded to length 6)
tag_lists = [
    [2, 4],         # Pop, Live
    [3, 6, 7],      # Rap, Dance, House
    [1, 5, 11]      # Rock, Indie, Alternative
]


# -- 1. Pad tag input (already correct) --
max_len = 6
tag_input = np.array([
    tags + [0] * (max_len - len(tags)) for tags in tag_lists
])

# -- 2. Scale numeric input once --
scaler = joblib.load("scaler.pkl")
new_numeric_scaled = scaler.transform(new_event_numeric)

# -- 3. Predict with correct input shapes --
pred_scaled = model.predict({
    "numeric_input": new_numeric_scaled,
    "tag_input": tag_input
}).flatten()

# -- 4. Denormalize --
pred_scores = pred_scaled * (y_max - y_min) + y_min

# -- 5. Print results --
for i, score in enumerate(pred_scores):
    print(f" Event → Predicted Success Score: {score:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
 Event → Predicted Success Score: 0.6641
 Event → Predicted Success Score: 0.7156
 Event → Predicted Success Score: 0.7414
